In [616]:
import pandas as pd
import json

pd.set_option('display.max_columns', None)
all_df = pd.read_csv('./data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv', encoding="ISO-8859-1", keep_default_na=False, dtype={'Item Code': str})
lat_lon_code = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv', keep_default_na=False)

lat_lon_code = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv')
lat_lon_code = lat_lon_code.sort_values(by=['Country']).drop_duplicates(subset=['Alpha-2 code'])
lat_lon_code.loc[:, "Numeric code"] = lat_lon_code["Numeric code"].str.strip(' "').astype(int)
lat_lon_code.loc[:, "Alpha-2 code"] = lat_lon_code["Alpha-2 code"].str.strip(' "')
lat_lon_code.loc[:, "Alpha-3 code"] = lat_lon_code["Alpha-3 code"].str.strip(' "')
lat_lon_code.loc[:, "Latitude (average)"] = lat_lon_code["Latitude (average)"].str.strip(' "').astype(float)
lat_lon_code.loc[:, "Longitude (average)"] = lat_lon_code["Longitude (average)"].str.strip(' "').astype(float)
area_code_to_iso_dict = lat_lon_code[["Numeric code", "Alpha-2 code"]].set_index("Numeric code")['Alpha-2 code'].to_dict()

all_df.loc[:, 'Area Code (M49)'] = all_df['Area Code (M49)'].str.strip(" '").astype(int)
all_df.loc[:, 'ISO-A2'] = all_df['Area Code (M49)'].replace(area_code_to_iso_dict)
all_df = all_df[all_df['ISO-A2'].isin(lat_lon_code['Alpha-2 code'])]

def make_year_cols(start, end):
    return [f'Y{y}' for y in range(start, end)]

In [617]:
all_df = all_df.merge(
    lat_lon_code[['Latitude (average)', 'Longitude (average)', 'Country', 'Alpha-2 code', 'Alpha-3 code']],
    how='left',
    left_on='ISO-A2',
    right_on='Alpha-2 code',
)

all_df = all_df.melt(
    id_vars=["Country", "ISO-A2", "Alpha-3 code", "Longitude (average)", "Latitude (average)", 'Item', 'Item Code', 'Element', 'Unit'], 
    value_vars=make_year_cols(2000, 2021), 
    value_name='Quantity', 
    var_name='Year'
)

all_df['Year'] = all_df['Year'].str.strip('Y').astype(int)
all_df['Quantity'] = all_df['Quantity'].replace({'': 0})
all_df['Quantity'] = all_df['Quantity'].astype(float)
all_df = all_df.rename(columns={"Country": "Area"})
all_df.loc[:, 'Area'] = all_df['Area'].apply(lambda x: x if not x.endswith('of') and not x.endswith('of the') else ' '.join(x.split(', ')[::-1]))

In [618]:
population = pd.read_csv('./data/population_by_country.csv')
population = population.melt(
    id_vars=['Country Name', 'Country Code'],
    value_vars=[str(y) for y in range(2000, 2021)],
    value_name='Population',
    var_name='Year',
)

population['Year'] = population['Year'].astype(int)
all_df = pd.merge(
    all_df, population,
    how='left',
    left_on=['Alpha-3 code', 'Year'],
    right_on=['Country Code', 'Year']
)

In [619]:
continent = pd.read_csv('./data/UNSD — Methodology.csv', sep=';', encoding='utf-8', keep_default_na=False)
all_df = all_df.merge(
    continent[['Region Name', 'ISO-alpha2 Code']],
    how='left',
    left_on='ISO-A2',
    right_on='ISO-alpha2 Code'
).rename(columns={'Region Name': 'Continent'}).drop(columns=['ISO-alpha2 Code'])

# Codes

In [620]:
codes = pd.read_csv('./data/cpc_codes.csv', encoding='ISO-8859-1', dtype={'CPC21code': str})

In [621]:
code_to_type_lookup = {
    '011': 'Cereals',
    '012': 'Vegetables',
    '013': 'Fruits and nuts',
    '211': 'Meat and meat products',
    '22': 'Dairy and egg'
}

manual_code_to_item = {
    '211': ['Meat of camels, fresh or chilled', 'Meat of cattle with the bone, fresh or chilled', 'Meat of pig with the bone, fresh or chilled', 'Meat of asses, fresh or chilled', 'Meat of mules, fresh or chilled', 'Meat of other domestic camelids, fresh or chilled', 'Meat of other domestic rodents, fresh or chilled', 'Meat of pigeons and other birds n.e.c., fresh, chilled or frozen'] + ['Edible offal of cattle, fresh, chilled or frozen', 'Edible offal of goat, fresh, chilled or frozen', 'Edible offal of sheep, fresh, chilled or frozen', 'Edible offal of pigs, fresh, chilled or frozen', 'Edible offal of buffalo, fresh, chilled or frozen'],
    '22': ['Raw milk of camel', 'Raw milk of cattle', 'Raw milk of goats', 'Raw milk of sheep', 'Raw milk of buffalo', 'Hen eggs in shell, fresh', 'Eggs from other birds in shell, fresh, n.e.c.'],
}

In [622]:
for code, code_type in code_to_type_lookup.items():
    if code in manual_code_to_item:
        all_df.loc[all_df['Item'].isin(manual_code_to_item[code]), 'Item Hierarchy Type'] = code_type
    else:
        all_item_types = codes[codes['CPC21code'].str.startswith(code)]['CPC21title']
        all_df.loc[all_df['Item'].isin(all_item_types), 'Item Hierarchy Type'] = code_type

In [623]:
all_df[all_df['Item Hierarchy Type'] == 'Dairy and egg']['Element'].unique()

array(['Yield', 'Production', 'Laying', 'Milk Animals'], dtype=object)

# check country name

In [24]:
with open('./data/ne_110m.json', 'r') as f:
    text = f.read()

In [ ]:
for country_name in all_df['Area'].unique():
    if text.find('"NAME":"{}"'.format(country_name)) == -1:
        print(country_name)


# wrangle file

In [624]:
select_columns = [
    'Area',
    'ISO-A2',
    "Unit",
    'Latitude (average)',
    'Longitude (average)',
    'Continent',
    'Year',
    'Population',
    'Item Hierarchy Type'
]

remove_if_null_in_subset = ["ISO-A2", "Continent", "Quantity", "Item Hierarchy Type"]

In [685]:
df = all_df[
    (all_df['Element'] == 'Production') &\
    (all_df['Unit'] == 'tonnes')
].copy()
df.loc[:, 'Quantity'] = df['Quantity'].fillna(0) # so sum groupby keeps
df = df.groupby([
    'Area',
    'ISO-A2',
    'Item Hierarchy Type',
    'Latitude (average)',
    'Longitude (average)',
    'Continent',
    'Population',
    'Year',
    'Element',
    'Unit',
])['Quantity'].sum().reset_index().dropna(
  axis=0,
  how='any',
  subset=remove_if_null_in_subset
)[[
    'Area',
    'ISO-A2',
    'Item Hierarchy Type',
    'Latitude (average)',
    'Longitude (average)',
    'Continent',
    'Population',
    'Year',
    'Quantity'
]]

df.to_csv('data/processed/production_major_categories_per_country_2000-2020.csv', index=False)

In [684]:
production_elements = ['Area harvested', 'Producing Animals/Slaughtered', 'Milk Animals', 'Laying']
df = all_df[all_df['Element'].isin(production_elements + ['Production'])]
df = df.dropna(subset=remove_if_null_in_subset, how='any')

count_producing_unit = df[df['Element'].isin(production_elements)].copy()
count_producing_unit['Quantity'] = count_producing_unit['Quantity'].fillna(0) # aggregate by sum so set to 0
count_producing_unit = count_producing_unit.groupby(['ISO-A2', 'Year', 'Element', 'Unit', 'Item Hierarchy Type'])['Quantity'].sum().reset_index()
count_producing_unit['Quantity no multiplier'] = count_producing_unit['Quantity'] * 999 * count_producing_unit['Unit'].str.startswith('1000') + count_producing_unit['Quantity']
count_producing_unit['Unit no multiplier'] = count_producing_unit['Unit'].str.replace('1000 ', '')
assert count_producing_unit.notna().all(axis=0).all()

total_producing_unit = count_producing_unit.groupby(['ISO-A2', 'Year', 'Unit no multiplier', 'Item Hierarchy Type'])['Quantity no multiplier'].sum().reset_index()
total_prod = df[(df['Element'] == 'Production') & (df['Unit'] == 'tonnes')].groupby(['ISO-A2', 'Year', 'Item Hierarchy Type'])['Quantity'].sum().reset_index()
total_producing_unit.rename(columns={"Quantity no multiplier": "Producing unit quantity", "Unit no multiplier": "Producing unit"}, inplace=True)
total_prod.rename(columns={"Quantity": "Production quantity"}, inplace=True)

merged = pd.merge(
    total_producing_unit, total_prod,
    on=['ISO-A2', 'Year', 'Item Hierarchy Type']
)
merged['Quantity'] = merged['Production quantity'] / merged['Producing unit quantity']
merged = merged.drop(columns=['Producing unit quantity', 'Production quantity', 'Producing unit'])
merged = merged.merge(
    df[['ISO-A2', 'Area', 'Year', 'Continent', 'Population', 'Latitude (average)', 'Longitude (average)']].drop_duplicates(), 
    how='left',
    on=['ISO-A2', 'Year']
)
merged = merged.dropna(subset=remove_if_null_in_subset, how='any')
merged.to_csv('./data/processed/yield_major_categories_per_country_2000-2020.csv', index=False)

total_producing_unit_globe = total_producing_unit.groupby(['Year', 'Item Hierarchy Type'])['Producing unit quantity'].sum().reset_index()
total_prod_glob = total_prod.groupby(['Year', 'Item Hierarchy Type'])['Production quantity'].sum().reset_index()

average_yield = pd.merge(
    total_producing_unit_globe, total_prod_glob,
    on=['Year', 'Item Hierarchy Type']
)

def year_on_year(df: pd.DataFrame):
    df = df.set_index('Year').sort_index().copy()
    df['Quantity'] = df['Production quantity'] / df['Producing unit quantity']
    return (df['Quantity'] - df['Quantity'].shift(1))

growth = average_yield.groupby(['Item Hierarchy Type']).apply(year_on_year).reset_index().drop(columns=[2000]).melt(id_vars=['Item Hierarchy Type']).rename(columns={"value": "yield growth"})
average_yield = average_yield.merge(
    growth,
    how='left',
    on=['Year', 'Item Hierarchy Type']
)
average_yield.to_csv('./data/processed/production_yield_global_2000_2020.csv', index=False)

In [656]:
average_yield.shift(1)

,Year,Item Hierarchy Type,Producing unit quantity,Production quantity,Quantity
0,NaN,NaN,NaN,NaN,NaN
1,2000.0,Cereals,6.655916e+08,2.046690e+09,3.074993
2,2000.0,Dairy and egg,5.704988e+09,6.329329e+08,0.110944
3,2000.0,Fruits and nuts,5.031708e+07,4.417205e+08,8.778740
4,2000.0,Meat and meat products,3.676797e+09,1.607517e+08,0.043721
...,...,...,...,...,...
100,2019.0,Vegetables,2.848049e+07,7.781348e+08,27.321675
101,2020.0,Cereals,7.218593e+08,2.980950e+09,4.129545
102,2020.0,Dairy and egg,8.586382e+09,9.741701e+08,0.113455
103,2020.0,Fruits and nuts,6.615674e+07,7.088890e+08,10.715295


In [647]:
total_producing_unit_globe

Year  Item Hierarchy Type   
2000  Cereals                   6.655916e+08
      Dairy and egg             5.704988e+09
      Fruits and nuts           5.031708e+07
      Meat and meat products    3.676797e+09
      Vegetables                2.313421e+07
                                    ...     
2020  Cereals                   7.218593e+08
      Dairy and egg             8.586382e+09
      Fruits and nuts           6.615674e+07
      Meat and meat products    4.814223e+09
      Vegetables                2.873227e+07
Name: Quantity, Length: 105, dtype: float64

In [688]:
df = all_df[
    (all_df["Element"] == "Production") &\
    (all_df["Unit"] == 'tonnes') &\
    # (all_df["Year"] == '2020')
].dropna(
  axis=0,
  how='any',
  subset=remove_if_null_in_subset,
).groupby(["Item", "Item Hierarchy Type", "Year"])[["Quantity"]]\
 .sum()\
 .reset_index()

df.to_csv('./data/processed/production_all_category_2020.csv', index=False)